In [ ]:
import glob
import numpy as np
import pandas as pd
import re
import os
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

import nest_asyncio
import asyncio
nest_asyncio.apply()

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from ast import Slice

# For converting 1 level

class leveldata2csv_converter:

    def __init__(self, _text_path):
        self.text_path =_text_path

        self.sl = self.covert_leveltext_series()
        self.df = self.find_headers()
        self.final_df = pd.DataFrame()


        # convert lvl txt to series
    def covert_leveltext_series(self):
        f = open(self.text_path, "r")
        data = f.readlines()
        # print(data)
        f.close()
        sl = pd.Series(data)
        return sl

    #finding headers
    def find_headers1(self):
        s_headline = pd.DataFrame()
        print(self.sl.to_string())
        pattern = r"\[# ----- \| #-----]"
        found = re.findall(pattern, self.sl.to_string())
        print(found)
        # found = self.sl.str.contains('# -----')
        s_headline = self.sl[found==True]
        s_headline = s_headline.reset_index()
        return s_headline

    def find_headers(self):
        s_headline = pd.DataFrame()
        # print(self.sl.to_string())
        pattern = r"#\s*-----\s*"  # Matches '# -----' or '#-------'
        found = [bool(re.search(pattern, str(x))) for x in self.sl]
        # print(found)
        # found = self.sl.str.contains('# -----')
        s_headline = self.sl[found]
        s_headline = s_headline.reset_index()
        return s_headline

    # exract level number
    def get_lvl_num(self):
        w = self.df[0][0].split()
        if 'Level'in w:
            index = w.index('Level')
            d = {'Level' : [int(w[index+1])]}
            lvl_no_df = pd.DataFrame(d)
            # lvl_no_df['Level'] = lvl_no_df['Level'].astype(int)
            return lvl_no_df
        elif 'Pack'in w:
            index = w.index('Pack')
            d = {'Level' : [int(w[index+1])]}
            lvl_no_df = pd.DataFrame(d)
            return lvl_no_df

        else:
            return "Not Exist ", self.df[0][0]


    # separating key ,value in list
    def convert_kv_list_working(self,k1, k2):

        d = self.df[self.df[0].str.contains(k1)]

        if len(d) == 0:
            d = self.df[self.df[0].str.contains(k2)]

        if len(d)>0:

            header_start_idx = d.index[0]
            header_end_idx = header_start_idx+1

            t_ser = self.sl[(self.df['index'][header_start_idx]):(self.df['index'][header_end_idx])]
            t_ser = t_ser.apply(lambda x : x[:-1])
            t_ser = t_ser[t_ser!='']
            t_ser = t_ser[t_ser.str.contains('=')]
            t_ser = t_ser.reset_index()
            t_ser = t_ser[0].str.split('=')
            return t_ser
        else:
            return pd.Series()


    # separating key ,value in list
    def convert_kv_list(self,k1, k2):

        d = self.df[self.df[0].str.contains(k1)]

        # print('d: ',d)

        if len(d) == 0:
            d = self.df[self.df[0].str.contains(k2)]

        #print(d[0].str.contains(k1))
        #print(d[0].str.contains(k2))

        if (d[0].str.contains(k1).empty & d[0].str.contains(k2).empty):
            return pd.Series()

        if len(d)>0:

            header_start_idx = d.index[0]
            header_end_idx = header_start_idx+1

            t_ser = self.sl[(self.df['index'][header_start_idx]):(self.df['index'][header_end_idx])]

            t_ser = t_ser.apply(lambda x : x[:-1])
            t_ser = t_ser[t_ser!='']
            t_ser = t_ser[t_ser.str.contains('=')]
            t_ser = t_ser.reset_index()
            t_ser = t_ser[0].str.split('=')
            return t_ser
        else:
            return pd.Series()


    # converting key value as dataframe
    def convert_list_df(self, k1, k2):

        t_ser = self.convert_kv_list(k1, k2)

        if not t_ser.empty:
            df = pd.DataFrame()
            df['key']=t_ser.apply(lambda x : x[0])
            df['value']=t_ser.apply(lambda x : x[1])
            return df
        else:
            return pd.DataFrame()

    def str2list(self,x):
        if '.'in x:
            return list(x)
        else :
            return x.split(",")


    # convert collection item to df
    def get_collection_item_df_old(self, k1, k2):

        prefixlist = ['avatar','sprite','type','useArchFlyTo','rotateOnFlyTo','flyToUI']

        col_df = self.convert_list_df(k1, k2)
        if not col_df.empty:
            is_present = ~col_df['key'].str.contains('|'.join(prefixlist))
            result_df = col_df[is_present]

            df = result_df
            # converting to list
            df['value'] = df['value'].apply(self.str2list)

            df = df.explode('value', ignore_index=True)

            # Extract the numeric part from the 'key' column
            df['key'] = df['key'] +'_'+ df.groupby('key').cumcount().astype(str)

            # Filter rows where values in 'Column1' are numeric
            numeric_filter = pd.to_numeric(df['value'], errors='coerce').notna()
            df = df[numeric_filter]

            #df['value'] = df['value'].apply(lambda x: 0 if x == '.' else x)

            df = df.set_index('key')
            df = df.T
            df.reset_index(drop=True, inplace=True)

            return df
        else:
            return pd.DataFrame()

     # convert collection item to df
    def get_collection_item_df(self, k1, k2):

        prefixlist = ['color','goal']

        col_df = self.convert_list_df(k1, k2)

        if not col_df.empty:
            is_present = col_df['key'].str.contains('|'.join(prefixlist))
            result_df = col_df[is_present]

            df = result_df
            # converting to list
            df['value'] = df['value'].apply(self.str2list)

            df = df.explode('value', ignore_index=True)

            # Extract the numeric part from the 'key' column
            df['key'] = df['key'] +'_'+ df.groupby('key').cumcount().astype(str)

            # Filter rows where values in 'Column1' are numeric
            numeric_filter = pd.to_numeric(df['value'], errors='coerce').notna()
            df = df[numeric_filter]


    #         df['value'] = df['value'].apply(lambda x: 0 if x == '.' else x)

            df = df.set_index('key')
            df = df.T
            df.reset_index(drop=True, inplace=True)

            return df
        else:
            return pd.DataFrame()

    # converting gameboard items to dataframe
    def convert_gameboard_item_df(self,k1, k2):

        df = self.convert_list_df(k1, k2)

        if not df.empty:

            item_df = pd.DataFrame()
            df[(df['key']=='new')]['value']
            d = (df[(df['key']=='new')]['value']).reset_index()
            item_list = list(d['value'][0])

            weight_df = df[(df['key']=='weights')]
            weight_df = weight_df.reset_index()
            weight_list = (weight_df['value'][0]).split(' ')

            item_df['new'] = [x+'_weights' for x in item_list]
            item_df['weights']= weight_list[:len(item_df['new'])]
            item_df = item_df.set_index('new')

            item_df = item_df.T
            item_df.reset_index(drop=True, inplace=True)

            # Weights
            item_df=item_df.replace('', 0)
            item_df=item_df.replace('18s', 18)

            return item_df
        else:
            return pd.DataFrame()

    # converting gameboard items to dataframe
    def convert_gameboard_item_df_new(self,k1, k2):

        df = self.convert_list_df(k1, k2)

#         item_df = pd.DataFrame()
        #df[('new' in df['key'] )]['value']
#         d = (df[('new' in df['key'])]['value']) #.reset_index()
#         item_list = list(d['value'][0])

#         weight_df = df[('weights' in df['key'])]
#         weight_df = weight_df.reset_index()
#         weight_list = (weight_df['value'][0]).split(' ')

#         item_df['new'] = [x+'_weights' for x in item_list]
#         item_df['weights']= weight_list[:len(item_df['new'])]
#         item_df = item_df.set_index('new')

#         item_df = item_df.T
#         item_df.reset_index(drop=True, inplace=True)

#         # Weights
#         item_df=item_df.replace('', 0)
#         item_df=item_df.replace('18s', 18)

        return df

    def convert_gameboard_item_df22(self, k1, k2):
        # Assuming self.convert_list_df() returns the DataFrame with 'key' and 'value' columns
        df = self.convert_list_df(k1, k2)

        if not df.empty:
            # Use str.contains to filter rows based on a regex pattern
            mask_new = df['key'].str.contains('new[0-9]?', regex=True)
            d = df[mask_new].reset_index(drop=True)

            # Extract values from the DataFrame
            item_list = list(d['value'])

            # Use str.contains to filter rows based on the 'weights[0-9]?' pattern
            mask_weights = df['key'].str.contains('weights[0-9]?', regex=True)
            weight_df = df[mask_weights].reset_index(drop=True)
            weight_list = weight_df['value'][0].split(' ')

            # Create the item DataFrame
            item_df = pd.DataFrame({'new': [x + '_weights' for x in item_list], 'weights': weight_list})
            item_df['weights'] = item_df['weights'].replace('', 0).astype(int)
            item_df = item_df.set_index('new')

            item_df = item_df.T
            item_df.reset_index(drop=True, inplace=True)

            # Weights
            item_df = item_df.replace('', 0)
            item_df = item_df.replace('18s', 18)

            return item_df
        else:
            return pd.DataFrame()


    def split_level(self,x):
        a = x.split(' ')
        a = [word for word in a if "#" not in word]
        a = [word for word in a if word.strip()]
        if len(a) > 1:
            return a
        else:
            return ''

    def level_wincondition_df(self,k1, k2):

        df = self.convert_list_df(k1, k2)

        if not df.empty:

            unwanted_featurs = ['levelPrefabs']
            is_present = ~df['key'].str.contains('|'.join(unwanted_featurs))
            df = df[is_present]
            df['value'] = df['value'].apply(self.split_level)
            df = df[df['value'] != '']

            k = df['value'].apply(lambda x: x[0])
            v = df['value'].apply(lambda x: x[1])

            d = {}
            for i in range(len(k.values)):
                d[k.values[i]] = [v.values[i]]

            df1 = pd.DataFrame(d)
            return  df1

        else:
            return pd.DataFrame()


    def split(self,x):
        a = x.split(' ')
        if len(a) > 1:
            return a[1]
        return x


    def convert_kv_df(self,k1, k2):

        df = self.convert_list_df(k1, k2)

        if not df.empty:

            df['value'] = df['value'].apply(self.split)

            # adding index with string value
            n_key = []
            for i in range(0,len(df['key'])):
                n_key.append(df['key'][i]+str(i))

            df['n_key'] = n_key
            df = df[['n_key', 'value' ]]
            df['n_key'] = df['n_key'].astype(str)

            if k1 == 'Glinda the Good':
                prefixlist = ['avatar','sprite','type','useArchFlyTo','rotateOnFlyTo','flyToUI']
                is_present = ~df['n_key'].str.contains('|'.join(prefixlist))
                result_df = df[is_present]
                df = result_df

            df = df[~df['n_key'].str.contains('#')]
            df = df.set_index('n_key')

            df = df.T
            df.reset_index(drop=True, inplace=True)

            return  df
        else:
            return pd.DataFrame()

    # To Test
    def get_gameboard_df(self, k1, k2):
        df = self.convert_list_df(k1, k2)
        if not df.empty:
            return df
        else:
            return pd.DataFrame()
    def find_words_with_last_letter_plus(self,df):
        words_with_plus = []
        for column in df.columns:
          if column.endswith('+'):
            words_with_plus.append(column)
        return words_with_plus

    def find_pattern(self, df,pattern):
        unwanted_columns = []
        for col in df.columns:
          if re.findall(pattern, col):
            unwanted_columns.append(col)
        return unwanted_columns

    def convert_gameboard_df(self, k1, k2):

        # Gameboard to df
        df = self.convert_list_df(k1, k2)
        #-------

        # Handling upper case and lower case issue
        df.loc[df['key'] != 'row', 'value'] = df.loc[df['key'] != 'row', 'value'].map(lambda x: x.lower() if isinstance(x, str) else x)
        # print(df)
        #-------
        if not df.empty :

            n_df = pd.DataFrame()

            arr = df['key'].unique()

            for item in arr:
            # print(item)
                w = item

                rowdf = df[df['key']== w]
                # print(rowdf)

                # converting to 1D array
                if len(rowdf)> 2:
                    s = ''
                    for i in rowdf['value']:
                        s += i

                    # converting to df
                    arr_df = pd.DataFrame()
                    arr_df[w+'Key'] = [w + str(x+1) for x in range(len(s))]
                    arr_df[w+'Value'] = list(s)

                    # converting one hot encoding
                    encoded_df = pd.get_dummies(arr_df, columns = [arr_df.columns[1]])

                    cols = encoded_df.columns.values

                    # converting into 1 row
                    m_df = pd.melt(encoded_df, id_vars=[cols[0]], value_vars=cols[1:(len(cols))])

                    m_df['new_var'] = m_df[w+'Key'] +'_'+ m_df['variable']
                    m_df = m_df[['new_var','value']]
                    m_df = m_df.set_index('new_var')

                    ddf = m_df.T

                    ddf.reset_index(drop=True, inplace=True)

                    # Checking and removing  columns ends with plus sign
                    plus_col = self.find_words_with_last_letter_plus(ddf)
                    # print(plus_col)
                    if len(plus_col)>0 :
                        ddf.drop(columns = plus_col, inplace=True)

                    # Checking and removing in columns
                    pattern_in =r'_inValue_'
                    in_col = self.find_pattern(ddf, pattern_in)
                    # print(dot_col)
                    if len(in_col)>0 :
                        ddf.drop(columns = in_col, inplace=True)

                    # Checking and removing out columns
                    pattern_out =r'_outValue_'
                    out_col = self.find_pattern(ddf, pattern_out)
                    # print(dot_col)
                    if len(out_col)>0 :
                        ddf.drop(columns = out_col, inplace=True)

                    # Checking and removing hash columns
                    pattern_hash =r'^#.'
                    hash_col = self.find_pattern(ddf, pattern_hash)
                    # print(dot_col)
                    if len(hash_col)>0 :
                        ddf.drop(columns = hash_col, inplace=True)

                    # checking and removing dot columns
                    pattern=r'\w+\.'
                    dot_col = self.find_pattern(ddf, pattern)
                    # print(dot_col)
                    if len(dot_col)>0 :
                        ddf.drop(columns = dot_col, inplace=True)

                    # concatinating
                    n_df = pd.concat([n_df, ddf], axis=1, join='outer')

            return n_df
        else:
            return pd.DataFrame()

    def concatinate_df(self, df):
        if not df.empty:
            return pd.concat([self.final_df, df], axis=1, join='outer')
        else:
            return self.final_df


    def lvl2df_converter(self):

        #concatinating level number to final df  # good
        level_num_df = self.get_lvl_num()
        print(level_num_df)
        self.final_df = self.concatinate_df(level_num_df)

        #concatinating gameboard item to df  # good
        gameboard_item_df = self.convert_gameboard_item_df('Gameboard Items', ' ')
        self.final_df = self.concatinate_df(gameboard_item_df)

        #concatinating level to df # good
        lvl_df = self.level_wincondition_df('Level','')
        self.final_df = self.concatinate_df( lvl_df)

        # concatinating collection to final df  # Good
        col_item_df = self.get_collection_item_df('Collectable Items', ' ')
        self.final_df = self.concatinate_df( col_item_df)

        # concatinating Gameboard to df # GOOD # ITS GAME BOARD
        # gameboard_df = self.convert_gameboard_df('The Gameboard', 'the rack')
        # self.final_df = self.concatinate_df( gameboard_df)

#_________ NOT Required ____________________________________
          #concatinating star to df  # this is not required
#         star_df = self.convert_kv_df('Star/Points Targets','')
#         self.final_df = self.concatinate_df( star_df)

          #concatinating power to df # # NOT Required                        #2
#         powerup_df = self.convert_kv_df('Power Ups','',)
#         powerup_df = powerup_df.replace('', np.nan)
#         self.final_df = self.concatinate_df( powerup_df)

#         concatinating glinda to df   # # Required # I dont take this
#         glinda_df = self.convert_kv_df('Glinda the Good','',)
#         self.final_df = self.concatinate_df( glinda_df)

        return self.final_df

##
class Special_Level_Handler:

    def __init__(self, _text_path):
        self.text_path =_text_path
    #self.spl_levels_path = []
    #self.sl = self.is_find_scroll_level()
    #self.df = self.find_headers()

     # convert level txt to series
    def is_find_scroll_level(self):
        with open(self.text_path, "r") as f:
            data = f.readlines()

        sl = pd.Series(data)
        found = sl.str.contains('panelsNum=').any()  # Use .any() to check if any line contains 'panelsNum='

        if found:
            return self.text_path
        else:
            return None

    def is_puzzlium_level(self):
        with open(self.text_path, "r") as f:
            data = f.readlines()

        sl = pd.Series(data)
        found = sl.str.contains('Puzzlium').any()  # Use .any() to check if any line contains 'panelsNum='

        if found:
            return self.text_path
        else:
            return None


# For merging all level
class leveldf_merger:
    def __init__(self, _level_path):
        self.leveltext_list = sorted(glob.glob(_level_path))
        self.all_lvl_df = pd.DataFrame()
        self.special_level_list = []
        self.puzzlium_level_list = []

    #Concatinating level by level
    def merge_lvls(self, start_idx, end_idx):
        for lvl_path in self.leveltext_list[start_idx : end_idx]:
            lvl_converter = leveldata2csv_converter(lvl_path)
            lvl_df = lvl_converter.lvl2df_converter()
            self.all_lvl_df = pd.concat([self.all_lvl_df, lvl_df], axis= 0, ignore_index=True)
        return self.all_lvl_df

    def find_Spl_levels(self, start_idx, end_idx):
        for lvl_path in self.leveltext_list[start_idx : end_idx]:
            level_handler = Special_Level_Handler(lvl_path)
            is_spl_level = level_handler.is_find_scroll_level()
            if is_spl_level:
                self.special_level_list.append(lvl_path)
        print(self.special_level_list)

    def find_puzzlium_levels(self, start_idx, end_idx):
        for lvl_path in self.leveltext_list[start_idx : end_idx]:
            level_handler = Special_Level_Handler(lvl_path)
            is_puzz_level = level_handler.is_puzzlium_level()
            if is_puzz_level:
                self.puzzlium_level_list.append(lvl_path)
        print(self.puzzlium_level_list)

    def save_spl_lvl_path(self, path):
        df = pd.DataFrame({"lvls":self.special_level_list})
        df.to_csv(path)

    def save_puzzlium_lvl_path(self, path):
        df = pd.DataFrame({"lvls":self.puzzlium_level_list})
        df.to_csv(path)

    #finding headers
    async def handle_scroll_levels(self, spl_lvl_csv_path):
        spl_lvl_paths_df = pd.read_csv(spl_lvl_csv_path)

        for spl_lvl_path in spl_lvl_paths_df['lvls']:
            data = []
            lvl_num = spl_lvl_path.split("/")[-1].split('.')[0]

            #Replacing temp levels with scroll level
            f_temp = open("/Users/vakumar/Documents/Personal/Py_Projects/Win_Rate_Prediction/Data/temp_level.txt", "r")
            temp_data = f_temp.readlines()
            f_temp.close()

            new_data = []
            new_data.append(f"# ----- Level {lvl_num} ---------------------------")
            new_data += temp_data

            #Writing back to scroll levels
            with open(str(spl_lvl_path), "w") as f:
                f.writelines(new_data)
            await asyncio.sleep(2)
            print(lvl_num)

    async def handle_puzzlium_levels(self, puzz_lvl_csv_path):
        puzz_lvl_paths_df = pd.read_csv(puzz_lvl_csv_path)

        for puzz_lvl_path in puzz_lvl_paths_df['lvls']:
#             data = []
            lvl_num = puzz_lvl_path.split("/")[-1].split('.')[0]

#             #Replacing temp levels with scroll level
#             f_temp = open("/Users/vakumar/Documents/Personal/Py_Projects/Win_Rate_Prediction/Data/temp_level.txt", "r")
#             temp_data = f_temp.readlines()
#             f_temp.close()

#             new_data = []
#             new_data.append(f"# ----- Level {lvl_num} ---------------------------")
#             new_data += temp_data

#             #Writing back to scroll levels
#             with open(str(spl_lvl_path), "w") as f:
#                 f.writelines(new_data)
#             await asyncio.sleep(2)
#             print(lvl_num)

            f_temp = open(puzz_lvl_path, "r")
            temp_data = f_temp.readlines()
            f_temp.close()

            data_df = pd.DataFrame(temp_data)

            data_df['check'] =data_df[0].str.contains('Puzzlium')
            data_df = data_df[data_df['check']==False]
            new_data =list(data_df[0])

            #Writing back to puzz levels
            with open(str(puzz_lvl_path), "w") as f:
                f.writelines(new_data)
            await asyncio.sleep(1)
            print(lvl_num)


In [ ]:
# 6399 Levels

strt = 0
end = 6400


# strt = 1230
# end = 1231

lvl_path_text ='/content/drive/MyDrive/Colab Notebooks/winrate/6400_levels/6400 Levels/*.txt'

lvl_merger = leveldf_merger(lvl_path_text)
all_df = lvl_merger.merge_lvls(strt,end)


all_df.fillna(0, inplace=True)
# all_df = all_df.astype(int)

# all_df.to_excel("Lvls_1000-1100_10oct24.xlsx")
all_df.to_csv("/content/drive/MyDrive/Colab Notebooks/winrate/gm_levels/Lvls_1-6400_no-gameboard_data_241114.csv")

print(all_df.shape)
all_df.head()

Streaming output truncated to the last 5000 lines.
0   3901
   Level
0   3902
   Level
0   3903
   Level
0   3904
   Level
0   3905
   Level
0   3906
   Level
0   3907
   Level
0   3908
   Level
0   3909
   Level
0   3910
   Level
0   3911
   Level
0   3912
   Level
0   3913
   Level
0   3914
   Level
0   3915
   Level
0   3916
   Level
0   3917
   Level
0   3918
   Level
0   3919
   Level
0   3920
   Level
0   3921
   Level
0   3922
   Level
0   3923
   Level
0   3924
   Level
0   3925
   Level
0   3926
   Level
0   3927
   Level
0   3928
   Level
0   3929
   Level
0   3930
   Level
0   3931
   Level
0   3932
   Level
0   3933
   Level
0   3934
   Level
0   3935
   Level
0   3936
   Level
0   3937
   Level
0   3938
   Level
0   3939
   Level
0   3940
   Level
0   3941
   Level
0   3942
   Level
0   3943
   Level
0   3944
   Level
0   3945
   Level
0   3946
   Level
0   3947
   Level
0   3948
   Level
0   3949
   Level
0   3950
   Level
0   3951
   Level
0   3952
   Level
0   3953
   L

,Level,r_weights,b_weights,g_weights,y_weights,moves,target.yellow.goal_0,v_weights,target.blue.goal_0,target.green.goal_0,target.red.goal_0,o_weights,target.orange.goal_0,w_weights,target.drop1.goal_0,c_weights,score,#target.yellow.goal_0,target.violet.goal_0,h_weights,target.hay.goal_0,target.flower.goal_0,p_weights,# target.green.goal_0,target.smoke.goal_0,target.drop.goal_0,z_weights,target.bones.goal_0,target.red_2.goal_0,target.Bones.goal_0,t_weights,target.tire.goal_0,target.Hay.goal_0,frogs,bugs,target.Violet.goal_0,#target.green.goal_0,u_weights,winkies,#target.red.goal_0,path,#target.bones.goal_0,ice,target.collect4.goal_0,m_weights,target.munchkins.goal_0,torch,f_weights,movabledraggers,#target.flower.goal_0,target.hay.goal _0,target.tire.goal _0,bigMover,#target.hay.goal_0,x_weights,movableDraggers,target.fireball.goal_0,target.colorizeR.goal_0,target.colorizeB.goal_0,target.colorizeO.goal_0,a_weights,target.colorizeV.goal_0,target.colorizeY.goal_0,target.colorizeG.goal_0,HelpRovers,mining,e_weights,target.energy.goal_0,bigmover,target.witchball.goal_0,#target.energy.goal_0,l_weights,target.munchkinegg.goal_0,q_weights,target.bouquet.goal_0,target.munchkinegg.color_0,colorswitch.target.r.pos_0,colorswitch.target.r.pos_1,colorswitch.target.g.pos_0,colorswitch.target.g.pos_1,_weights,BigMover,colorswitch.target.b.pos_0,colorswitch.target.b.pos_1,colorswitch.target.y.pos_0,colorswitch.target.y.pos_1,colorswitch,Path
0,1,25,25,25,25,10,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,24,30,23,23,10,0,0,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,20,20,20,40,10,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,18,18,30,17,10,0,17,0,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,18,18,18,28,10,21,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 6399 Levels

strt = 4000
end = 5000


# strt = 1230
# end = 1231

lvl_path_text ='/content/drive/MyDrive/Colab Notebooks/winrate/6400_levels/6400 Levels/*.txt'

lvl_merger = leveldf_merger(lvl_path_text)
all_df = lvl_merger.merge_lvls(strt,end)


all_df.fillna(0, inplace=True)
all_df = all_df.astype(int)

# all_df.to_excel("Lvls_1000-1100_10oct24.xlsx")
all_df.to_csv("/content/drive/MyDrive/Colab Notebooks/winrate/gm_levels/Lvls_4k-5k.csv")

print(all_df.shape)
all_df.head()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:

# from zipfile import ZipFile

# def unzip_file(zip_file_path, extract_path):
#   """Unzips a zip file to a specified directory.

#   Args:
#     zip_file_path: The path to the zip file.
#     extract_path: The path to the directory where the contents should be extracted.
#   """
#   with ZipFile(zip_file_path, 'r') as zipObj:
#     # Extract all the contents of zip file in different directory
#     zipObj.extractall(extract_path)


# Example usage:
# zip_file_path = '/content/my_zip_file.zip'  # Replace with your zip file path
# extract_path = '/content/extracted_files'   # Replace with the desired extraction directory
# unzip_file(zip_file_path, extract_path)

In [ ]:
# zip_file_path = '//content/drive/MyDrive/Colab Notebooks/winrate/6400 Levels.zip'
# extract_path = '/content/drive/MyDrive/Colab Notebooks/winrate/6400_levels'   # Replace with the desired extraction directory
# unzip_file(zip_file_path, extract_path)

In [ ]:
# # 6399 Levels

# strt = 933
# end = 934


# # strt = 1230
# # end = 1231

# lvl_path_text ='/content/drive/MyDrive/Colab Notebooks/winrate/6400_levels/6400 Levels/*.txt'

# lvl_merger = leveldf_merger(lvl_path_text)
# all_df = lvl_merger.merge_lvls(strt,end)


# all_df.fillna(0, inplace=True)
# all_df = all_df.astype(int)

# # all_df.to_excel("Lvls_1000-1100_10oct24.xlsx")
# all_df.to_csv("/content/drive/MyDrive/Colab Notebooks/winrate/gm_levels/Lvls_0-1k.csv")

# print(all_df.shape)
# all_df.head()

   Level
0    934
(1, 1)


,Level
0,934


In [ ]:
(1, 1297)

(1, 1297)

In [ ]:
all_df.columns

Index(['Level', 'row1_rowValue_.', 'row2_rowValue_.', 'row3_rowValue_.',
       'row4_rowValue_.', 'row5_rowValue_.', 'row6_rowValue_.',
       'row7_rowValue_.', 'row8_rowValue_.', 'row9_rowValue_.',
       ...
       'sludge72_sludgeValue_x', 'sludge73_sludgeValue_x',
       'sludge74_sludgeValue_x', 'sludge75_sludgeValue_x',
       'sludge76_sludgeValue_x', 'sludge77_sludgeValue_x',
       'sludge78_sludgeValue_x', 'sludge79_sludgeValue_x',
       'sludge80_sludgeValue_x', 'sludge81_sludgeValue_x'],
      dtype='object', length=5187)

In [ ]:
# def find_pattern(pattern):
#   unwanted_columns = []
#   for col in all_df.columns:
#     if re.findall(pattern, col):
#       unwanted_columns.append(col)
#   return unwanted_columns


# arr = pd.Series(find_pattern(pattern = r'bones'))
# # arr

#   # find duplicates in new columns and create dup series
# dup = arr[arr.duplicated(keep='first')]
# len(arr),len(dup)

In [ ]:
(2430, 0),(1620, 0)

In [ ]:
# text_path ='/content/drive/MyDrive/Colab Notebooks/winrate/levels/0240.txt'

# f = open(text_path, "r")
# data = f.readlines()
# print(data)
# f.close()
# sl = pd.Series(data)
# sl

In [ ]:
all_df

""


In [ ]:
csv_lvl_files = ['Lvls_0-1k','Lvls_1k-2k','Lvls_2k-3k','Lvls_3k-4k','Lvls_4k-5k','Lvls_5k-64k']


In [ ]:
gm_lvl_df = pd.DataFrame()

for i in csv_lvl_files:
    df = pd.read_csv(f"/content/drive/MyDrive/Colab Notebooks/winrate/gm_levels/{i}.csv")
    gm_lvl_df = pd.concat([gm_lvl_df, df], axis= 0, ignore_index=True)

gm_lvl_df.to_csv("/content/drive/MyDrive/Colab Notebooks/winrate/gm_levels_1-6400_241111.csv")

In [ ]:
gm_lvl_df.head()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
gm_lvl_df.shape

(6400, 65971)

In [ ]:
gm_lvl_df.to_csv("/content/drive/MyDrive/Colab Notebooks/winrate/gm_levels_1-6400.csv")

In [ ]:
pd.read_csv("/content/drive/MyDrive/Colab Notebooks/winrate/gm_levels/Lvls_6k-64k.csv")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# all_df.to_excel("Lvls_1000-1100_10oct24.xlsx")
all_df.to_csv("/content/drive/MyDrive/Colab Notebooks/winrate/gm_levels/Lvls_1-1000.csv")

In [ ]:
# import regex as re

# f_temp = open("/Users/vakumar/Documents/Personal/Py_Projects/Win_Rate_Prediction/Data/6400 Levels/0963.txt", "r")
# temp_data = f_temp.readlines()
# f_temp.close()

# data_df = pd.DataFrame(temp_data)

# # found =data_ser.str.contains('Puzzlium').any()
# # found

# data_df['check'] =data_df[0].str.contains('Puzzlium')
# data_df = data_df[data_df['check']==False]
# new_data =list(data_df[0])

# puzz_lvl_path = 'puss_test_lvl.txt'

# #Writing back to scroll levels
# with open(str(puzz_lvl_path), "w") as f:
#     f.writelines(new_data)
# # await asyncio.sleep(1)
# # print(lvl_num)

In [ ]:
data_df['check'] is not False

True

In [ ]:


# 1000 Levels

# strt = 1
# end = 6400

# lvl_path_text = "/Users/vakumar/Documents/Personal/Py_Projects/Win_Rate_Prediction/Data/6400 Levels/*.txt"

# lvl_merger = leveldf_merger(lvl_path_text)
# # lvl_merger.find_Spl_levels(strt,end)
# # lvl_merger.save_spl_lvl_path('spl_levels.csv')

# # # Handle scroll level
# # # lvl_merger.handle_scroll_levels('spl_levels.csv')

# asyncio.run(lvl_merger.handle_scroll_levels('spl_levels.csv'))



In [ ]:
# 1000 puzzilium Levels

# strt = 1
# end = 6400

# lvl_path_text = "/Users/vakumar/Documents/Personal/Py_Projects/Win_Rate_Prediction/Data/6400 Levels/*.txt"

# lvl_merger = leveldf_merger(lvl_path_text)
# # lvl_merger.find_puzzlium_levels(strt,end)
# # lvl_merger.save_puzzlium_lvl_path('Puzzlium_levels.csv')



# asyncio.run(lvl_merger.handle_puzzlium_levels('Puzzlium_levels.csv'))





In [ ]:
df = all_df

In [ ]:
d1 =df[df['key'].str.contains('new')]
d2 =df[df['key'].str.contains('weights')]
# d = pd.concat([d1, d2], axis= 0, ignore_index=True)

d1

,Level,key,value
0,1231.0,new0,rybgv
7,NaN,new1,rybg
13,NaN,new2,rybgv


In [ ]:
d2

,Level,key,value
6,NaN,weights0,25 25 25 25 0
12,NaN,weights1,25 25 25 25
19,NaN,weights2,20 20 20 20 20


In [ ]:
item_df = pd.DataFrame()
item_df['key'] = d1['key']
item_df['value'] = d1['value']

item_df['key1'] = d2['key']
# item_df['value1'] = d2['value']

item_df

,key,value,key1
0,new0,rybgv,NaN
7,new1,rybg,NaN
13,new2,rybgv,NaN


In [ ]:
item_list = list(d['value'][0])

In [ ]:
item_list

['r', 'y', 'b', 'g', 'v']

In [ ]:
df[df['key'].str.contains('weights')]

,Level,key,value
6,NaN,weights0,25 25 25 25 0
12,NaN,weights1,25 25 25 25
19,NaN,weights2,20 20 20 20 20


In [ ]:
item_df = pd.DataFrame()

d = (df[('new' in df['key'])]['value']) #.reset_index()
item_list = list(d['value'][0])

weight_df = df[('weights' in df['key'])]
weight_df = weight_df.reset_index()
weight_list = (weight_df['value'][0]).split(' ')

item_df['new'] = [x+'_weights' for x in item_list]
item_df['weights']= weight_list[:len(item_df['new'])]
item_df = item_df.set_index('new')

item_df = item_df.T
item_df.reset_index(drop=True, inplace=True)

# Weights
item_df=item_df.replace('', 0)
item_df=item_df.replace('18s', 18)

In [ ]:
# 1000 Levels

lvl_path_text = "/Users/vakumar/Documents/Personal/Py_Projects/Win_Rate_Prediction/Data/LEVELS/*.txt"

lvl_merger = leveldf_merger(lvl_path_text)
all_df = lvl_merger.merge_lvls(1230,1231)


# all_df.fillna(0, inplace=True)
# all_df = all_df.astype(int)




print(all_df.shape)
all_df

(20, 3)


,Level,key,value
0,1231.0,new0,rybgv
1,NaN,color0.r,Red
2,NaN,color0.y,Yellow
3,NaN,color0.b,Blue
4,NaN,color0.g,Green
5,NaN,color0.v,Violet
6,NaN,weights0,25 25 25 25 0
7,NaN,new1,rybg
8,NaN,color1.r,Red
9,NaN,color1.y,Yellow


In [ ]:
# for power ups

# encoded_df = pd.get_dummies(all_df,
#                             columns = all_df.columns[1:],
#                             drop_first=True)
# encoded_df

In [ ]:
# encoded_df.to_excel("Lvls_1000_powerups_v1.xlsx")

In [ ]:
col = []

for i in all_df.columns:
    print('--')
    print(i)
    print(all_df[i].unique())
    print(all_df[i].dtype)
    if len(all_df[i].unique())>1:
        col.append(i)


In [ ]:
df = pd.DataFrame()
df.isna()

""


In [ ]:
s = pd.Series()
s.empty

True

In [ ]:
col

In [ ]:
all_df['row18_rowValue_b'].unique()

In [ ]:
all_df_n  = all_df[col]
all_df_n.shape

In [ ]:
all_df_n.to_excel("Lvls_1000_gameboard_v2.xlsx")

In [ ]:
# encoded_df = pd.get_dummies(all_df[['preLevelPowerup1','inLevelPowerup2']],
#                             columns = ['preLevelPowerup1','inLevelPowerup2'],
#                            drop_first=True)
# encoded_df

In [ ]:
'glindaLevel0','target.glinda.isWinCondition7'

In [ ]:
# all_df.T.to_excel("Lvls_1000_gameboard_v1.xlsx")

### TESTING

In [ ]:
lvl_path_text = "/Users/vakumar/Documents/Personal/Py_Projects/Win_Rate_Prediction/Data/LEVELS/*.txt"
lvl_converter = leveldata2csv_converter(sorted(glob.glob(lvl_path_text))[1229])


In [ ]:
# level number to final df
lvl = lvl_converter.get_lvl_num()
print(lvl)
lvl

   Level
0   1230


,Level
0,1230


In [ ]:

# collection to final df
col_df = lvl_converter.get_collection_item_df('Collectable Items', ' ')
print(col_df.shape)
col_df

----
0    0
Name: index, dtype: int64
d size 0
header_start_idx  0


KeyError: 1

In [ ]:
# gameboard item to df
gb_items_df = lvl_converter.convert_gameboard_item_df('Gameboard Items', ' ')
print(gb_items_df.shape)
gb_items_df

In [ ]:
# clevel to df
level_df = lvl_converter.convert_kv_df('Level','')
# print(level_df.shape)
level_df

In [ ]:
# star to df
start_df = lvl_converter.convert_kv_df('Star/Points Targets','')
print(start_df.shape)
start_df

In [ ]:
#  power to df
pwr_df = lvl_converter.convert_kv_df('Power Ups','',)
print(pwr_df.shape)
pwr_df

In [ ]:
#  glinda to df
glinda = lvl_converter.convert_kv_df('Glinda the Good','',)
print(glinda.shape)
glinda

In [ ]:
#  Gameboard to df
gb_df = lvl_converter.convert_gameboard_df('The Gameboard', 'the rack')
print(gb_df.shape)
gb_df

In [ ]:
# 1 level

lvl_path_text = "/Users/vakumar/Documents/Personal/Py_Projects/Win_Rate_Prediction/Data/WozMM Levels 1-1000/*.txt"

lvl_merger = leveldf_merger(lvl_path_text)
all_df = lvl_merger.merge_lvls(0,1)


print(all_df.shape)
all_df


In [ ]:
# 5 Levels

lvl_path_text = "/Users/vakumar/Documents/Personal/Py_Projects/Win_Rate_Prediction/Data/WozMM Levels 1-1000/*.txt"

lvl_merger = leveldf_merger(lvl_path_text)
all_df = lvl_merger.merge_lvls(0,5)

print(all_df.shape)
all_df


In [ ]:
# 1000 Levels

lvl_path_text = "/Users/vakumar/Documents/Personal/Py_Projects/Win_Rate_Prediction/Data/WozMM Levels 1-1000/*.txt"

lvl_merger = leveldf_merger(lvl_path_text)
all_df = lvl_merger.merge_lvls(0,1000)

print(all_df.shape)
all_df



In [ ]:
# all_df.T.to_excel("Lvls_1000_preproc_v2.xlsx")